# Story 3

*Do stricter gun laws reduce firearm gun deaths?*

In [71]:
import requests
import pandas as pd
import numpy as np
import json
import seaborn as sns
import geopandas as gpd


ImportError: cannot import name 'Point' from 'shapely' (/Users/andrewbowen/opt/anaconda3/lib/python3.9/site-packages/shapely/__init__.py)

In [52]:
response = requests.get("https://data.cdc.gov/resource/489q-934x.json").json()
# response = requests.get("https://data.cdc.gov/resource/bi63-dtpu.json?cause_name=Firearm-related injury")

## Datasources

- [NCHS Leading Cause of Deathe API Endpoint](https://dev.socrata.com/foundry/data.cdc.gov/bi63-dtpu)

In [53]:
df = pd.json_normalize(response)
df.head()

,year_and_quarter,time_period,cause_of_death,rate_type,unit,rate_overall,rate_sex_female,rate_sex_male,rate_alaska,rate_alabama,...,rate_age_1_4,rate_age_5_14,rate_age_15_24,rate_age_25_34,rate_age_35_44,rate_age_45_54,rate_age_55_64,rate_65_74,rate_age_75_84,rate_age_85_plus
0,2020 Q1,12 months ending with quarter,All causes,Age-adjusted,"Deaths per 100,000",718.6,605.1,851,691.2,898.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020 Q1,12 months ending with quarter,Alzheimer disease,Age-adjusted,"Deaths per 100,000",29.8,33.6,23.8,24.1,44.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020 Q1,12 months ending with quarter,COVID-19,Age-adjusted,"Deaths per 100,000",1.7,1.2,2.3,NaN,0.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020 Q1,12 months ending with quarter,Cancer,Age-adjusted,"Deaths per 100,000",145.7,126,172.2,141.3,159.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020 Q1,12 months ending with quarter,Chronic liver disease and cirrhosis,Age-adjusted,"Deaths per 100,000",11.5,8.1,15.2,16,13.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
df.columns

Index(['year_and_quarter', 'time_period', 'cause_of_death', 'rate_type',
       'unit', 'rate_overall', 'rate_sex_female', 'rate_sex_male',
       'rate_alaska', 'rate_alabama', 'rate_arkansas', 'rate_arizona',
       'rate_california', 'rate_colorado', 'rate_connecticut',
       'rate_district_of_columbia', 'rate_delaware', 'rate_florida',
       'rate_georgia', 'rate_hawaii', 'rate_iowa', 'rate_idaho',
       'rate_illinois', 'rate_indiana', 'rate_kansas', 'rate_kentucky',
       'rate_louisiana', 'rate_massachusetts', 'rate_maryland', 'rate_maine',
       'rate_michigan', 'rate_minnesota', 'rate_missouri', 'rate_mississippi',
       'rate_montana', 'rate_north_carolina', 'rate_north_dakota',
       'rate_nebraska', 'rate_new_hampshire', 'rate_new_jersey',
       'rate_new_mexico', 'rate_nevada', 'rate_new_york', 'rate_ohio',
       'rate_oklahoma', 'rate_oregon', 'rate_pennsylvania',
       'rate_rhode_island', 'rate_south_carolina', 'rate_south_dakota',
       'rate_tennessee', 'ra

## Data Wrangling

In [55]:
# Pulling out only firearm deaths that are age-adjusted
df = df.loc[(df.cause_of_death == "Firearm-related injury") & (df.rate_type == "Age-adjusted") & (df.time_period=="3-month period")]

In [56]:
state_cols = ['year_and_quarter', 'rate_alaska', 'rate_alabama', 'rate_arkansas', 'rate_arizona',
       'rate_california', 'rate_colorado', 'rate_connecticut',
       'rate_district_of_columbia', 'rate_delaware', 'rate_florida',
       'rate_georgia', 'rate_hawaii', 'rate_iowa', 'rate_idaho',
       'rate_illinois', 'rate_indiana', 'rate_kansas', 'rate_kentucky',
       'rate_louisiana', 'rate_massachusetts', 'rate_maryland', 'rate_maine',
       'rate_michigan', 'rate_minnesota', 'rate_missouri', 'rate_mississippi',
       'rate_montana', 'rate_north_carolina', 'rate_north_dakota',
       'rate_nebraska', 'rate_new_hampshire', 'rate_new_jersey',
       'rate_new_mexico', 'rate_nevada', 'rate_new_york', 'rate_ohio',
       'rate_oklahoma', 'rate_oregon', 'rate_pennsylvania',
       'rate_rhode_island', 'rate_south_carolina', 'rate_south_dakota',
       'rate_tennessee', 'rate_texas', 'rate_utah', 'rate_virginia',
       'rate_vermont', 'rate_washington', 'rate_wisconsin',
       'rate_west_virginia', 'rate_wyoming', 'rate_overall']
df = df[state_cols].transpose()
df.head()

,31,75,119,163,207,251,295,339,383,427,471,515
year_and_quarter,2020 Q1,2020 Q2,2020 Q3,2020 Q4,2021 Q1,2021 Q2,2021 Q3,2021 Q4,2022 Q1,2022 Q2,2022 Q3,2022 Q4
rate_alaska,21.4,20.2,26.4,26.1,19.4,28.8,24.8,27.9,22.5,20.4,24.5,NaN
rate_alabama,23,23.4,24.9,23,26.4,27.6,26.4,25.2,24.1,26.8,28,NaN
rate_arkansas,20.8,27.1,21.3,21.1,22.4,27.2,21.1,22.4,19.2,25,23.4,NaN
rate_arizona,15.9,15.4,18.4,16.9,16.7,18.4,19,19.1,18.7,22.7,20.8,NaN


In [57]:
df.columns = df.iloc[0]

In [58]:
df = df.iloc[1:]
df.head()

year_and_quarter,2020 Q1,2020 Q2,2020 Q3,2020 Q4,2021 Q1,2021 Q2,2021 Q3,2021 Q4,2022 Q1,2022 Q2,2022 Q3,2022 Q4
rate_alaska,21.4,20.2,26.4,26.1,19.4,28.8,24.8,27.9,22.5,20.4,24.5,NaN
rate_alabama,23,23.4,24.9,23,26.4,27.6,26.4,25.2,24.1,26.8,28,NaN
rate_arkansas,20.8,27.1,21.3,21.1,22.4,27.2,21.1,22.4,19.2,25,23.4,NaN
rate_arizona,15.9,15.4,18.4,16.9,16.7,18.4,19,19.1,18.7,22.7,20.8,NaN
rate_california,7.5,7.9,9.2,9.4,8.7,9.1,9.5,8.5,8.3,8.9,9.4,NaN


In [59]:
df.index

Index(['rate_alaska', 'rate_alabama', 'rate_arkansas', 'rate_arizona',
       'rate_california', 'rate_colorado', 'rate_connecticut',
       'rate_district_of_columbia', 'rate_delaware', 'rate_florida',
       'rate_georgia', 'rate_hawaii', 'rate_iowa', 'rate_idaho',
       'rate_illinois', 'rate_indiana', 'rate_kansas', 'rate_kentucky',
       'rate_louisiana', 'rate_massachusetts', 'rate_maryland', 'rate_maine',
       'rate_michigan', 'rate_minnesota', 'rate_missouri', 'rate_mississippi',
       'rate_montana', 'rate_north_carolina', 'rate_north_dakota',
       'rate_nebraska', 'rate_new_hampshire', 'rate_new_jersey',
       'rate_new_mexico', 'rate_nevada', 'rate_new_york', 'rate_ohio',
       'rate_oklahoma', 'rate_oregon', 'rate_pennsylvania',
       'rate_rhode_island', 'rate_south_carolina', 'rate_south_dakota',
       'rate_tennessee', 'rate_texas', 'rate_utah', 'rate_virginia',
       'rate_vermont', 'rate_washington', 'rate_wisconsin',
       'rate_west_virginia', 'rate_wyom

In [60]:
# Clean up state names and set index
df["state"] = [s.replace("_", " ").replace("rate", "").title() for s in df.index.values]
df = df.set_index('state')
df.head()

year_and_quarter,2020 Q1,2020 Q2,2020 Q3,2020 Q4,2021 Q1,2021 Q2,2021 Q3,2021 Q4,2022 Q1,2022 Q2,2022 Q3,2022 Q4
state,,,,,,,,,,,,
Alaska,21.4,20.2,26.4,26.1,19.4,28.8,24.8,27.9,22.5,20.4,24.5,NaN
Alabama,23,23.4,24.9,23,26.4,27.6,26.4,25.2,24.1,26.8,28,NaN
Arkansas,20.8,27.1,21.3,21.1,22.4,27.2,21.1,22.4,19.2,25,23.4,NaN
Arizona,15.9,15.4,18.4,16.9,16.7,18.4,19,19.1,18.7,22.7,20.8,NaN
California,7.5,7.9,9.2,9.4,8.7,9.1,9.5,8.5,8.3,8.9,9.4,NaN


## Geometry Data for Later Choropleths
Grabbed this GeoJSON from [plotly datasets](https://github.com/plotly/datasets)

In [61]:
# Read in GeoJSON containing state boundary data
geojson_path = "./data/us-states.json"
states = gpd.read_file("./data/us-states.json")
states['state'] = states['name']
states.head()

,id,name,geometry,state
0,AL,Alabama,"POLYGON ((-87.35930 35.00118, -85.60667 34.984...",Alabama
1,AK,Alaska,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691...",Alaska
2,AZ,Arizona,"POLYGON ((-109.04250 37.00026, -109.04798 31.3...",Arizona
3,AR,Arkansas,"POLYGON ((-94.47384 36.50186, -90.15254 36.496...",Arkansas
4,CA,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",California


In [72]:
df = df.reset_index()

# Convert to geopandas dataframe
gdf = gpd.GeoDataFrame(df)

gdf.head()

0                    Alaska
1                   Alabama
2                  Arkansas
3                   Arizona
4                California
5                  Colorado
6               Connecticut
7      District Of Columbia
8                  Delaware
9                   Florida
10                  Georgia
11                   Hawaii
12                     Iowa
13                    Idaho
14                 Illinois
15                  Indiana
16                   Kansas
17                 Kentucky
18                Louisiana
19            Massachusetts
20                 Maryland
21                    Maine
22                 Michigan
23                Minnesota
24                 Missouri
25              Mississippi
26                  Montana
27           North Carolina
28             North Dakota
29                 Nebraska
30            New Hampshire
31               New Jersey
32               New Mexico
33                   Nevada
34                 New York
35                  

In [69]:
# Join Firearm data to geometry data
gdf = gdf.merge(states, how="left", on="state")
gdf.head()

,state,2020 Q1,2020 Q2,2020 Q3,2020 Q4,2021 Q1,2021 Q2,2021 Q3,2021 Q4,2022 Q1,2022 Q2,2022 Q3,2022 Q4,id_x,name_x,geometry_x,id_y,name_y,geometry_y
0,Alaska,21.4,20.2,26.4,26.1,19.4,28.8,24.8,27.9,22.5,20.4,24.5,NaN,NaN,NaN,None,NaN,NaN,None
1,Alabama,23,23.4,24.9,23,26.4,27.6,26.4,25.2,24.1,26.8,28,NaN,NaN,NaN,None,NaN,NaN,None
2,Arkansas,20.8,27.1,21.3,21.1,22.4,27.2,21.1,22.4,19.2,25,23.4,NaN,NaN,NaN,None,NaN,NaN,None
3,Arizona,15.9,15.4,18.4,16.9,16.7,18.4,19,19.1,18.7,22.7,20.8,NaN,NaN,NaN,None,NaN,NaN,None
4,California,7.5,7.9,9.2,9.4,8.7,9.1,9.5,8.5,8.3,8.9,9.4,NaN,NaN,NaN,None,NaN,NaN,None


TODO

- Bring in shapefile/state location data
